In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# New Section

In [ ]:
import re
from googlesearch import search
import warnings
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup

# Take input a disease and return the content of wikipedia's infobox for that specific disease

def diseaseDetail(term):
    diseases=[term]
    ret=term+"\n"
    for dis in diseases:
        # search "disease wikipedia" on google 
        query = dis+' wikipedia'
        for sr in search(query,tld="co.in",stop=10,pause=0.5): 
            # open wikipedia link
            match=re.search(r'wikipedia',sr)
            filled = 0
            if match:
                wiki = requests.get(sr,verify=False)
                soup = BeautifulSoup(wiki.content, 'html5lib')
                # Fetch HTML code for 'infobox'
                info_table = soup.find("table", {"class":"infobox"})
                if info_table is not None:
                    # Preprocess contents of infobox
                    for row in info_table.find_all("tr"):
                        data=row.find("th",{"scope":"row"})
                        if data is not None:
                            symptom=str(row.find("td"))
                            symptom = symptom.replace('.','')
                            symptom = symptom.replace(';',',')
                            symptom = symptom.replace('<b>','<b> \n')
                            symptom=re.sub(r'<a.*?>','',symptom) # Remove hyperlink
                            symptom=re.sub(r'</a>','',symptom) # Remove hyperlink
                            symptom=re.sub(r'<[^<]+?>',' ',symptom) # All the tags
                            symptom=re.sub(r'\[.*\]','',symptom) # Remove citation text                     
                            symptom=symptom.replace("&gt",">")
                            ret+=data.get_text()+" - "+symptom+"\n"
#                            print(data.get_text(),"-",symptom)
                            filled = 1
                if filled:
                    break
    return ret

In [ ]:
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from statistics import mean
from nltk.corpus import wordnet 
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from itertools import combinations
from time import time
from collections import Counter
import operator
from xgboost import XGBClassifier
import math
#from Treatment import diseaseDetail
from sklearn.linear_model import LogisticRegression

warnings.simplefilter("ignore")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [ ]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
splitter = RegexpTokenizer(r'\w+')

In [ ]:
def synonyms(term):
    synonyms = []
    response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.content,  "html.parser")
    try:
        container=soup.find('section', {'class': 'MainContentContainer'}) 
        row=container.find('div',{'class':'css-191l5o0-ClassicContentCard'})
        row = row.find_all('li')
        for x in row:
            synonyms.append(x.get_text())
    except:
        None
    for syn in wordnet.synsets(term):
        synonyms+=syn.lemma_names()
    return set(synonyms)

In [ ]:
# Load Dataset scraped from NHP (https://www.nhp.gov.in/disease-a-z) & Wikipedia
# Scrapping and creation of dataset csv is done in a separate program
df_comb = pd.read_csv("/content/gdrive/MyDrive/dis_sym_dataset_comb.csv") # Disease combination
df_norm = pd.read_csv("/content/gdrive/MyDrive/dis_sym_dataset_norm.csv") # Individual Disease

X = df_comb.iloc[:, 1:]
Y = df_comb.iloc[:, 0:1]

In [ ]:
lr = LogisticRegression()
lr = lr.fit(X, Y)
scores = cross_val_score(lr, X, Y, cv=5)
# MLP Classifier
#from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), activation='tanh', solver='adam', max_iter=100)
mlp = mlp.fit(X, Y)
scores_mlp = cross_val_score(mlp, X, Y, cv=5)
# RF Classifier
rf = RandomForestClassifier(n_estimators=10, criterion='entropy')
rf = rf.fit(X, Y)
scores_rf = cross_val_score(rf, X, Y, cv=5)
# KNN Classifier
knn = KNeighborsClassifier(n_neighbors=31, weights='distance', n_jobs=4)
knn = knn.fit(X, Y)
scores_knn = cross_val_score(knn, X, Y, cv=5)
# DT Classifier
dt = DecisionTreeClassifier()
dt = dt.fit(X, Y)
scores_dt = cross_val_score(dt, X, Y, cv=5)
# SVM Classifier
svm = SVC()
svm = svm.fit(X, Y)
scores_svm = cross_val_score(svm, X, Y, cv=5)


In [ ]:
X = df_norm.iloc[:, 1:]
Y = df_norm.iloc[:, 0:1]
# List of symptoms
dataset_symptoms = list(X.columns)

In [ ]:
# Taking symptoms from user as input 
# coughing,pyrexia,tire,loss of smell
user_symptoms = str(input("Please enter symptoms separated by comma(,):\n")).lower().split(',')
# Preprocessing the input symptoms
processed_user_symptoms=[]
for sym in user_symptoms:
    sym=sym.strip()
    sym=sym.replace('-',' ')
    sym=sym.replace("'",'')
    sym = ' '.join([lemmatizer.lemmatize(word) for word in splitter.tokenize(sym)])
    processed_user_symptoms.append(sym)

Please enter symptoms separated by comma(,):
Increased thirst, Frequent urination,Extreme hunger, Unexplained weight loss


In [ ]:
# Taking each user symptom and finding all its synonyms and appending it to the pre-processed symptom string
user_symptoms = []
for user_sym in processed_user_symptoms:
    user_sym = user_sym.split()
    str_sym = set()
    for comb in range(1, len(user_sym)+1):
        for subset in combinations(user_sym, comb):
            subset=' '.join(subset)
            subset = synonyms(subset) 
            str_sym.update(subset)
    str_sym.add(' '.join(user_sym))
    user_symptoms.append(' '.join(str_sym).replace('_',' '))
# query expansion performed by joining synonyms found for each symptoms initially entered
##print("After query expansion done by using the symptoms entered")
#print(user_symptoms)

In [ ]:
# Loop over all the symptoms in dataset and check its similarity score to the synonym string of the user-input 
# symptoms. If similarity>0.5, add the symptom to the final list
found_symptoms = set()
for idx, data_sym in enumerate(dataset_symptoms):
    data_sym_split=data_sym.split()
    for user_sym in user_symptoms:
        count=0
        for symp in data_sym_split:
            if symp in user_sym.split():
                count+=1
        if count/len(data_sym_split)>0.5:
            found_symptoms.add(data_sym)
found_symptoms = list(found_symptoms)

In [ ]:
# Print all found symptoms
print("Symptoms that best match your search!")
for idx, symp in enumerate(found_symptoms):
    print(idx,":",symp)
    
# Show the related symptoms found in the dataset and ask user to select among them
select_list = input("\nPlease select all the relevant symptoms. Enter indices (separated-space):\n").split()

# Find other relevant symptoms from the dataset based on user symptoms based on the highest co-occurance with the
# ones that is input by the user
dis_list = set()
final_symp = [] 
counter_list = []
for idx in select_list:
    symp=found_symptoms[int(idx)]
    final_symp.append(symp)
    dis_list.update(set(df_norm[df_norm[symp]==1]['label_dis']))
   
for dis in dis_list:
    row = df_norm.loc[df_norm['label_dis'] == dis].values.tolist()
    row[0].pop(0)
    for idx,val in enumerate(row[0]):
        if val!=0 and dataset_symptoms[idx] not in final_symp:
            counter_list.append(dataset_symptoms[idx])

Symptoms that best match your search!
0 : increased hunger
1 : unexplained weight loss
2 : unintended weight loss
3 : frequent urination
4 : increased thirst
5 : red

Please select all the relevant symptoms. Enter indices (separated-space):
0 1 2 3 4


In [ ]:
# Symptoms that co-occur with the ones selected by user              
dict_symp = dict(Counter(counter_list))
dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)   
#print(dict_symp_tup) 

In [ ]:
# Iteratively, suggest top co-occuring symptoms to the user and ask to select the ones applicable 
found_symptoms=[]
count=0
for tup in dict_symp_tup:
    count+=1
    found_symptoms.append(tup[0])
    if count%5==0 or count==len(dict_symp_tup):
        print("\nCommon co-occuring symptoms:")
        for idx,ele in enumerate(found_symptoms):
            print(idx,":",ele)
        select_list = input("Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:\n").lower().split();
        if select_list[0]=='no':
            break
        if select_list[0]=='-1':
            found_symptoms = [] 
            continue
        for idx in select_list:
            final_symp.append(found_symptoms[int(idx)])
        found_symptoms = [] 


Common co-occuring symptoms:
0 : fever
1 : testicular pain
2 : fast heartbeat
3 : irritability
4 : muscle weakness
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
1

Common co-occuring symptoms:
0 : sleeping problem
1 : change bowel movement
2 : missed period
3 : nausea vomiting
4 : tender breast
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
no


In [ ]:
# Create query vector based on symptoms selected by the user
print("\nFinal list of Symptoms that will be used for prediction:")
sample_x = [0 for x in range(0,len(dataset_symptoms))]
for val in final_symp:
    print(val)
    sample_x[dataset_symptoms.index(val)]=1


Final list of Symptoms that will be used for prediction:
increased hunger
unexplained weight loss
unintended weight loss
frequent urination
increased thirst
testicular pain


In [ ]:
# Predict disease
#lr
lr = LogisticRegression()
lr = lr.fit(X, Y)
prediction_lr = lr.predict_proba([sample_x])
#mlp
mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), activation='tanh', solver='adam', max_iter=100)
mlp = mlp.fit(X, Y)
prediction_mlp = mlp.predict_proba([sample_x])
#rf
rf = RandomForestClassifier(n_estimators=10, criterion='entropy')
rf = rf.fit(X, Y)
prediction_rf = rf.predict_proba([sample_x])
# KNN 
knn = KNeighborsClassifier(n_neighbors=31, weights='distance', n_jobs=4)
knn = knn.fit(X, Y)
prediction_knn = knn.predict_proba([sample_x])
# DT Classifier
dt = DecisionTreeClassifier()
dt = dt.fit(X, Y)
prediction_dt = dt.predict_proba([sample_x])
# SVM Classifier
svm = SVC()
svm = svm.fit(X, Y)
prediction_svm = svm.predict([sample_x])

In [ ]:
# k = 4
# print("prediction_lr[0]:",prediction_lr[0].argsort()[-k:][::-1])
# print("prediction_mlp[0]:",prediction_mlp[0].argsort()[-k:][::-1])
# print("prediction_knn[0]:",prediction_knn[0].argsort()[-k:][::-1])
# print("prediction_dt[0]:",prediction_dt[0].argsort()[-k:][::-1])
# print("prediction_rf[0]:",prediction_rf[0].argsort()[-k:][::-1])
# print("prediction_svm:",prediction_svm)

In [ ]:
#mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), activation='tanh', solver='adam', max_iter=100)
#mlp = mlp.fit(X, Y)
#prediction = mlp.predict_proba([sample_x])

In [ ]:
k = 1
diseases = list(set(Y['label_dis']))
diseases.sort()
topk_lr = prediction_lr[0].argsort()[-k:][::-1]
topk_mlp = prediction_mlp[0].argsort()[-k:][::-1]
topk_knn = prediction_knn[0].argsort()[-k:][::-1]
topk_dt = prediction_dt[0].argsort()[-k:][::-1]
topk_rf = prediction_rf[0].argsort()[-k:][::-1]

In [ ]:
def diseasemap(topk):
  topk_dict = {}
  for idx,t in  enumerate(topk):
    match_sym=set()
    row = df_norm.loc[df_norm['label_dis'] == diseases[t]].values.tolist()
    row[0].pop(0)

    for idx,val in enumerate(row[0]):
        if val!=0:
            match_sym.add(dataset_symptoms[idx])
    prob = (len(match_sym.intersection(set(final_symp)))+1)/(len(set(final_symp))+1)
    prob *= mean(scores_mlp)
    topk_dict[t] = prob

  j = 0
  topk_sorted = dict(sorted(topk_dict.items(), key=lambda kv: kv[1], reverse=True))
  topk_index_mapping = []
  for key in topk_sorted:
    prob = topk_sorted[key]*100
    # print(str(j) + " Disease name:",diseases[key], "\tProbability:",str(round(prob, 2))+"%")
    topk_index_mapping.append(diseases[key])
    j += 1

  return topk_index_mapping

In [ ]:
pred = diseasemap(topk_knn)+diseasemap(topk_lr)+diseasemap(topk_rf)+diseasemap(topk_dt)+diseasemap(topk_mlp)+prediction_svm.tolist()
pred = Counter(pred)
predicted_disease = sorted(pred.items(), key=lambda kv:(kv[1], kv[0]),reverse = True)[:1][0][0]
print(f"\nTop disease predicted based on symptoms by 6 Models")
print(predicted_disease)


Top disease predicted based on symptoms by 6 Models
Diabetes Mellitus


In [ ]:
#print(f"\nTop disease predicted based on symptoms by 6 Models")
select = input("\nMore details about the disease? Enter '1' for continue or '-1' to discontinue and close the system:\n")
if select!='-1':
    print()
    print(diseaseDetail(predicted_disease))


More details about the disease? Enter '1' for continue or '-1' to discontinue and close the system:
1

Diabetes Mellitus
Pronunciation -       /  ˌ  d  aɪ       ə  ˈ  b  i       t  i  z  ,   -  t  ɪ  s  /     /  ˈ  m  ɛ  l       ɪ       t  ə  s  ,     m  ə  ˈ  l  aɪ  -/       
Specialty -  Endocrinology 
Symptoms -   
  Frequent urination 
 increased thirst 
 increased hunger    
  
Complications -  Diabetic ketoacidosis, hyperosmolar hyperglycemic state, heart disease, stroke, pain/pins and needles in hands and/or feet, chronic kidney failure, foot ulcers, cognitive impairment, gastroparesis   
Risk factors -    
Type 1 : Family history     
Type 2 : Obesity, lack of exercise, genetics,   
Diagnostic method -  High blood sugar   
Treatment -   
  Healthy diet 
 physical exercise    
  
Medication -  Insulin, anti-diabetic medication like metformin   
Frequency -  463 million (88%)   
Deaths -  42 million (2019)   

